In [40]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from google.colab import files
import io
import jaconv
import re

In [22]:
# 料理データセットの読み込み
uploaded = files.upload()  # ファイル選択ダイアログを出す
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))  # Excelファイルのシート名を確認
print(xls.sheet_names)  # Excelのシート名の確認

df_recipes_ori = pd.read_excel(xls, sheet_name='recipes')
df_prebiotics_ori = pd.read_excel(xls, sheet_name='prebiotics')
df_probiotics_ori = pd.read_excel(xls, sheet_name='probiotics')

# 列をすべて表示
pd.set_option('display.max_columns', None)

# 行をすべて表示
#pd.set_option('display.max_rows', None)

Saving 料理名と食材・栄養素のデータセット.xlsx to 料理名と食材・栄養素のデータセット (1).xlsx
['recipes', 'prebiotics', 'probiotics']


In [23]:
df_recipes = df_recipes_ori.copy()
df_prebiotics = df_prebiotics_ori.copy()
df_probiotics = df_probiotics_ori.copy()

In [24]:
# プレバイオティクスのデータの確認
df_prebiotics.head()

,野菜類,豆類,フルーツ,海藻類,きのこ類,その他
0,アスパラガス,大豆,バナナ,わかめ,しいたけ,はちみつ
1,ブロッコリー,レンズ豆,りんご,昆布,えのきたけ,NaN
2,キャベツ,ひよこ豆,ベリー,ひじき,まいたけ,NaN
3,ねぎ,いんげん,キウイ,のり,えのき,NaN
4,ごぼう,えだまめ,NaN,NaN,NaN,NaN


# 料理データセットの前処理

In [25]:
# " Nan "の数の確認
df_recipes.isnull().sum().sum()

np.int64(7180)

In [26]:
# " Nan "の穴埋め
# 文字列型の欠損を " - "に変換する
object_cols = df_recipes.select_dtypes(['object']).columns
df_recipes[object_cols] = df_recipes[object_cols].fillna('-')

# 数値型の欠損を " 0 "に変換する
num_cols = df_recipes.select_dtypes(['number']).columns
df_recipes[num_cols] = df_recipes[num_cols].fillna(0)

In [27]:
# 変換後の" Nan "の数の確認
df_recipes.isnull().sum().sum()

np.int64(0)

In [28]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0,-,0.0


In [29]:
# 「・」「＿」「（）」以降の単語を削除する関数
def clean_foodname(name):
  """
  IN:鶏卵・全卵＿生
  OUT:鶏卵
  """
  # 「（～）」を削除
  name = re.sub("（[^）]*）", "", name)
  # 「・」「＿」以降も削除
  name = re.sub("[・＿].*", "", name)
  return name

In [30]:
# clean_foodname関数の実行
df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)

/tmp/ipython-input-1019100835.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)


In [31]:
# プレバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pre_word = df_prebiotics.stack().tolist() # データフレームの列をリスト化する、「nan」は除外する
# ラベル付け
def check_prebiotics(cell):
  """
  それぞれのレシピにプレバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中を判断する
  """
  for x in pre_word:
    if x in cell:
      return 1
  return 0

In [32]:
# check_prebiotics関数の実行
df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)

/tmp/ipython-input-3022261290.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)


In [33]:
# プロバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pro_word = df_probiotics.values.ravel().tolist() # データフレームの列をリスト化する

In [34]:
# プロバイオティクスのラベル付け
def check_probiotics(cell):
  """
  それぞれのレシピにプロバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中）を判断する
  """
  for x in pro_word:
    if x in cell:
      return 1
  return 0

In [35]:
# check_prebiotics関数の実行
# 食材の列の範囲（インデックス8～46までの列）
df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:43:2].applymap(check_probiotics).max(axis=1)

/tmp/ipython-input-1256241574.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:43:2].applymap(check_probiotics).max(axis=1)


# 腸活スコアの算出

In [36]:
# ①プレバイオティクス・プロバイオティクスの点数
# 30 / 100 点
# ------------------------------------------------------------------------------------------------------
# 各料理の食材にプレバイオティクス or プロバイオティクスが含まれているのかを判定し、30点満点で評価する
# ------------------------------------------------------------------------------------------------------

# プレプロスコア関数
def prepro_score(row):
  score = 0
  if row['プレバイオティクス（全体）'] == 1:             # プレバイオティクスが 1 （食材が含まれている）とき +15点 する
    score += 15
  if row['プロバイオティクス（全体）'] == 1:             # プロバイオティクスが 1 （食材が含まれている）とき +15点 する
    score += 15
  return score

# 関数の実行
df_recipes['プレプロスコア']  = df_recipes.apply(prepro_score, axis=1)

In [37]:
# ②野菜量
# 40 / 100 点
# ---------------------------------------------------------------------------
# 各料理が1日あたりの野菜目安（350g）の何％かを計算し、40点満点で評価する
# ---------------------------------------------------------------------------

# 野菜量スコア関数
def veg_score(veg_g, daily_target=350, max_points=40):
  frac = veg_g / daily_target
  frac_clip = 1.0 if frac > 1.0 else frac
  score = frac * max_points
  return round(score, 1) # 小数点第1位で返す

# 関数の実行
df_recipes['野菜量スコア'] = df_recipes.apply(lambda row: veg_score(row['野菜量[グラム]']), axis=1)

In [ ]:
# ③食材多様性指数
# 30 / 100 点
# ---------------------------------------------------------------------------
# 料理に使用した食材の多様性を計算し、30点満点で評価する
# ---------------------------------------------------------------------------

# 食材の多様性スコア関数
def shannon_wiener():
  grams
  total = grams.sum()
  if total == 0:
    return 0
  p = grams /total
  return -np.sum(p * np.log(p))

# 各料理のグラムのリストを作成する
gram_cols = [colum for colum in df_recipes.columns if 'グラム' in colum]

# 関数の実行
df_recipes['食材多様性指数'] = df_recipes.apply(shannon_wiener, axis=1)

In [41]:
# 食材データ（例）
data = {
    "食材": ["キャベツ", "にんじん", "じゃがいも", "鶏肉"],
    "グラム": [100, 50, 150, 200]
}
df = pd.DataFrame(data)

def shannon_wiener(grams):
    total = grams.sum()
    if total == 0:
        return 0
    p = grams / total
    return -np.sum(p * np.log(p))

# 実行
H = shannon_wiener(df["グラム"])
print("シャノン多様性指数:", H)

シャノン多様性指数: 1.2798542258336676


In [ ]:
df_recipes.head()